In [6]:
import jax.numpy as jnp
import jax
from jax import random, vmap, jit, lax

In [7]:
key = random.PRNGKey(42)
k, key = random.split(key, 2)

In [8]:
# a single impage
img = random.normal(k, shape=(32, 32, 3))
print(img.shape)

(32, 32, 3)


In [27]:
# padding
def img_to_patches(img, patch_radius):
    old_shape = img.shape
    img_new = jnp.pad(img, (( patch_radius,  patch_radius), ( patch_radius,  patch_radius), (0, 0)))
    def _get_patch(c0,c1):
        tmp = lax.dynamic_slice_in_dim(img_new, c0, 2*patch_radius, axis=0)
        return lax.dynamic_slice_in_dim(tmp, c1, 2*patch_radius, axis=1)
    get_patch = vmap(_get_patch,(None,0),0)
    get_patch = vmap(get_patch,(0,None),0)
    index = jnp.arange(0, old_shape[0])
    patches = get_patch(index, index)
    return patches
    
# extract_patch = img_new[
#     center[0] : 1 + center[0] + 2 * pad_width,
#     center[1] : 1 + center[1] + 2 * pad_width,
#     :,
# ]

# patch_radius = 2
# img_new = jnp.pad(img, ((pad, pad), (pad, pad), (0, 0)))
# center = (32, 32)
# extract_patch = img_new[
#     center[0] : 1 + center[0] + 2 * pad_width,
#     center[1] : 1 + center[1] + 2 * pad_width,
#     :,
# ]
# extract_patch.shape

In [28]:
patches = img_to_patches(img, patch_radius=2)
patches.squeeze().shape

ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: traced array with shape int32[].
pad_width argument of jnp.pad
The error occurred while tracing the function img_to_patches at /tmp/ipykernel_31645/2050411284.py:3 for jit. This concrete value was not available in Python because it depends on the value of the argument patch_radius.

See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.ConcretizationTypeError